In [1]:
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime

In [2]:
# создаем список df, удаляем ненужное
rules = list()
for filename in os.listdir('.'):
    if filename.endswith(".csv") and not filename.startswith('result'):
        df = pd.read_csv('./' + filename, sep=';', encoding='ISO-8859-1')
        df = df.drop(columns = ['Ãðóïïà', 'Äàòà íà÷àëà äåéñòâèÿ', 'Äàòà îêîí÷àíèÿ äåéñòâèÿ', 'Àâòîð', 'Òèï ïðàâèëà', 'Ôîðìóëà'])
        rules.append(df)
# если первый файл содержит слово MIR, меняем порядок
if re.search('MIR', rules[0].to_string()) != None:
  rules.reverse()

In [3]:
# переименовываем столбцы
for df in rules:
  df.columns = ['id', 'risk', 'desc']

In [4]:
# указываем какие правила блокирующие
for df in rules:
  df['desc'] = np.where((df.risk == 100), 'Блок '+ df['desc'], df['desc'])

In [5]:
# создаем словари для импортируемых и копируемых правил, делаем понятные описания
channels = {'AlienATM': 'Чужой АТМ', 'ATM': 'ATM', 'CHnP': 'Cardholder not present', 'CHP': 'Cardholder present', 'OwnATM': 'Свой АТМ','RiskCHnP': 'Cardholder not present Risk MCC', 'Rub': 'руб.'}
channels = {r"\b{}\b".format(k): v for k, v in channels.items()}
toCopy = {'2COPY Block 2xCP succes MPS': 'после 2х успешных Cardholder present', '2COPY 3xCP no PIN MPS': '3xCardholder present без ввода пин-кода', '2COPY Country change': 'Смена страны', '2COPY Fallback': '1хFallback', '2COPY Block frequent AZS': '3хЗаправка', '2COPY 3Block ruchnoy vvod': '3хРучной ввод данных', '2COPY MPS Block EPOS rost summy Risk MCC USD': 'E-POS $ Risk MCC увеличение суммы', '2COPY Block 2xCP succes MIR': 'после 2х успешных Cardholder present', '2COPY 3xCP no PIN MIR': '3xCardholder present без ввода пин-кода', '2COPY MIR Block EPOS rost summy Risk MCC USD': 'E-POS $ Risk MCC увеличение суммы', '2COPY Block EPOS rost summy Risk MCC USD': 'E-POS $ Risk MCC увеличение суммы', '2COPY Block POS': 'после 2х успешных Cardholder present', '2COPY Card present no PIN Code': '3xCardholder present без ввода пин-кода', '2COPY Block EPOS rost summy Risk MCC RUB': 'E-POS руб Risk MCC увеличение суммы'}
toCopy = {r"\b{}\b".format(k): v for k, v in toCopy.items()}

for df in rules:
  df["desc"] = df["desc"].replace(channels, regex=True)
  df["desc"] = df["desc"].replace(toCopy, regex=True)

In [6]:
# делаем итоговую таблицу, объединяем номера правил
if len(rules) == 2:
  df_itog = pd.merge(rules[0], rules[1], how='inner', on = 'desc')
  df_itog = df_itog.assign(Nomer = df_itog.id_x.astype(str) + '/' + df_itog.id_y.astype(str))
if len(rules) == 1:
   df_itog = rules[0]

In [7]:
# удаляем лишнее из итоговой таблицы, причесываем, экспортируем
if len(rules) == 2:
  df_itog = df_itog.drop(columns = ['id_x', 'id_y', 'risk_x', 'risk_y'])
  df_itog.columns = ['Описание', 'Номер правила']
  df_itog = df_itog.reindex(['Номер правила', 'Описание'], axis=1)
# ищем правила, не попавшие в итоговый список, добавляем их
  for df in rules:
    df_missed = df[~df['desc'].isin(df_itog['Описание'])]
    df_missed = df_missed.drop(columns = 'risk')
    df_missed.columns = ['Номер правила', 'Описание']
    df_itog = pd.concat([df_itog, df_missed])
if len(rules) == 1:
  df_itog = df_itog.drop(columns = 'risk')
  df_itog.columns = ['Номер правила', 'Описание']
fileName = datetime.now().isoformat(sep='_', timespec='minutes')
df_itog.to_csv('result_' + fileName + '.csv', sep=';', encoding='cp1251', index=False)